## Pairwise comparisions optimization

In [1]:
import os
import warnings
from itertools import combinations

import numpy as np
import torch

# Suppress potential optimization warnings for cleaner notebook
warnings.filterwarnings("ignore")

SMOKE_TEST = os.environ.get("SMOKE_TEST")

In [2]:
# data generating helper functions
def utility(X):
    """Given X, output corresponding utility (i.e., the latent function)"""
    # y is weighted sum of X, with weight sqrt(i) imposed on dimension i
    weighted_X = X * torch.sqrt(torch.arange(X.size(-1), dtype=torch.float) + 1)
    y = torch.sum(weighted_X, dim=-1)
    return y

def generate_data(n, dim=2):
    """Generate data X and y"""
    # X is randomly sampled from dim-dimentional unit cube
    # we recommend using double as opposed to float tensor here for
    # better numerical stability
    X = torch.rand(n, dim, dtype=torch.float64)
    y = utility(X)
    return X, y


def generate_comparisons(y, n_comp, noise=0.1, replace=False):
    """Create pairwise comparisons with noise"""
    # generate all possible pairs of elements in y
    all_pairs = np.array(list(combinations(range(y.shape[0]), 2)))
    # randomly select n_comp pairs from all_pairs
    comp_pairs = all_pairs[np.random.choice(range(len(all_pairs)), n_comp, replace=replace)]
    # add gaussian noise to the latent y values
    c0 = y[comp_pairs[:, 0]] + np.random.standard_normal(len(comp_pairs)) * noise
    c1 = y[comp_pairs[:, 1]] + np.random.standard_normal(len(comp_pairs)) * noise
    reverse_comp = (c0 < c1).numpy()
    comp_pairs[reverse_comp, :] = np.flip(comp_pairs[reverse_comp, :], 1)
    comp_pairs = torch.tensor(comp_pairs).long()

    return comp_pairs

In [3]:
n = 50 if not SMOKE_TEST else 5
m = 100 if not SMOKE_TEST else 10
dim = 2
noise = 0.1
train_X, train_y = generate_data(n, dim=dim)
train_comp = generate_comparisons(train_y, m, noise=noise)

In [4]:
train_comp[:10]

tensor([[ 0,  9],
        [ 1, 10],
        [14, 35],
        [37, 11],
        [30, 45],
        [49, 18],
        [29, 47],
        [ 8, 38],
        [12, 19],
        [25, 39]])

In [5]:
train_X

tensor([[0.9505, 0.4956],
        [0.5662, 0.8793],
        [0.5712, 0.5899],
        [0.4404, 0.9366],
        [0.6893, 0.3740],
        [0.6769, 0.4142],
        [0.4997, 0.5573],
        [0.3896, 0.3665],
        [0.8841, 0.1407],
        [0.6708, 0.6748],
        [0.5093, 0.2745],
        [0.2017, 0.8635],
        [0.3276, 0.9151],
        [0.2199, 0.9883],
        [0.8069, 0.9183],
        [0.3979, 0.3067],
        [0.0591, 0.5177],
        [0.8881, 0.6097],
        [0.2112, 0.0306],
        [0.5043, 0.8074],
        [0.0056, 0.0658],
        [0.2947, 0.0184],
        [0.2375, 0.4143],
        [0.9804, 0.7604],
        [0.6854, 0.9641],
        [0.2752, 0.4439],
        [0.9278, 0.2032],
        [0.0751, 0.0821],
        [0.7742, 0.7569],
        [0.9383, 0.5658],
        [0.7994, 0.5697],
        [0.7342, 0.0285],
        [0.6585, 0.1126],
        [0.0874, 0.5618],
        [0.3254, 0.5447],
        [0.0351, 0.8501],
        [0.9405, 0.1093],
        [0.8177, 0.9672],
        [0.7

In [6]:
train_y[:10]

tensor([1.6515, 1.8096, 1.4055, 1.7649, 1.2182, 1.2627, 1.2878, 0.9080, 1.0830,
        1.6252], dtype=torch.float64)

In [7]:
from botorch.models.pairwise_gp import PairwiseGP, PairwiseLaplaceMarginalLogLikelihood
from botorch.fit import fit_gpytorch_mll

model = PairwiseGP(train_X, train_comp)
mll = PairwiseLaplaceMarginalLogLikelihood(model.likelihood, model)
mll = fit_gpytorch_mll(mll)

KeyboardInterrupt: 

In [ ]:
from scipy.stats import kendalltau


# Kendall-Tau rank correlation
def eval_kt_cor(model, test_X, test_y):
    pred_y = model.posterior(test_X).mean.squeeze().detach().numpy()
    return kendalltau(pred_y, test_y).correlation

n_kendall = 1000 if not SMOKE_TEST else 10

test_X, test_y = generate_data(n_kendall, dim=dim)
kt_correlation = eval_kt_cor(model, test_X, test_y)

print(f"Test Kendall-Tau rank correlation: {kt_correlation:.4f}")

Test Kendall-Tau rank correlation: 0.9321


In [ ]:
from botorch.acquisition.preference import AnalyticExpectedUtilityOfBestOption
from botorch.optim import optimize_acqf


def init_and_fit_model(X, comp, state_dict=None):
    """Model fitting helper function"""
    model = PairwiseGP(X, comp)
    mll = PairwiseLaplaceMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)
    return mll, model


def make_new_data(X, next_X, comps, q_comp):
    """Given X and next_X,
    generate q_comp new comparisons between next_X
    and return the concatenated X and comparisons
    """
    # next_X is float by default; cast it to the dtype of X (i.e., double)
    next_X = next_X.to(X)
    next_y = utility(next_X)
    next_comps = generate_comparisons(next_y, n_comp=q_comp, noise=noise)
    comps = torch.cat([comps, next_comps + X.shape[-2]])
    X = torch.cat([X, next_X])
    return X, comps

In [ ]:
algos = ["EUBO", "rand"]

NUM_TRIALS = 5 if not SMOKE_TEST else 2
NUM_BATCHES = 30 if not SMOKE_TEST else 2

dim = 5
NUM_RESTARTS = 3
RAW_SAMPLES = 256 if not SMOKE_TEST else 8
q = 2  # number of points per query
q_comp = 1  # number of comparisons per query

# initial evals
best_vals = {}  # best observed values
for algo in algos:
    best_vals[algo] = []

# average over multiple trials
for i in range(NUM_TRIALS):
    torch.manual_seed(i)
    np.random.seed(i)
    data = {}
    models = {}

    # Create initial data
    init_X, init_y = generate_data(q, dim=dim)
    comparisons = generate_comparisons(init_y, q_comp, noise=noise)
    test_X, test_y = generate_data(1000, dim=dim)
    # X are within the unit cube
    bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])

    for algo in algos:
        best_vals[algo].append([])
        data[algo] = (init_X, comparisons)
        _, models[algo] = init_and_fit_model(init_X, comparisons)
        model = models[algo]

        best_next_y = utility(init_X).max().item()
        best_vals[algo][-1].append(best_next_y)

    # we make additional NUM_BATCHES comparison queries after the initial observation
    for j in range(1, NUM_BATCHES + 1):
        for algo in algos:
            if algo == "EUBO":
                # create the acquisition function object
                acq_func = AnalyticExpectedUtilityOfBestOption(pref_model=model)
                # optimize and get new observation
                next_X, acq_val = optimize_acqf(
                    acq_function=acq_func,
                    bounds=bounds,
                    q=q,
                    num_restarts=NUM_RESTARTS,
                    raw_samples=RAW_SAMPLES,
                )
            else:
                # randomly sample data
                next_X, _ = generate_data(q, dim=dim)

            # update data
            X, comps = data[algo]
            X, comps = make_new_data(X, next_X, comps, q_comp)
            data[algo] = (X, comps)

            # refit models
            model = models[algo]
            _, model = init_and_fit_model(X, comps, model.state_dict())
            models[algo] = model

            # record the best observed values so far
            max_val = utility(X).max().item()
            best_vals[algo][-1].append(max_val)

KeyboardInterrupt: ignored

## AX to perfom Factorial design

In [ ]:
!pip install ax-platform==0.1.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.5/499.5 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 KB 11.0 MB/s eta 0:00:00


In [ ]:
def func(x):
  return x[0]**2+x[1]**2

In [ ]:
from ax import *
import numpy as np
import pandas as pd

data=[func(x) for x in np.random.rand(50,2)*np.random.randint(0,10)]

ImportError: ignored

In [ ]:
def eval_func(parameters):
    x = parameters["x"]
    y = parameters["y"]

    eval=func([x,y])+np.random.rand(1)

    return {"func": (eval, 0.0)}

In [ ]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {
            "name": "x",
            "type": "range",
            "bounds": [-10.0, 10.0],
            "value_type": "float",  # Optional, defaults to inference from type of "bounds".
            "log_scale": False,  # Optional, defaults to False.
        },
        {
            "name": "y",
            "type": "range",
            "bounds": [-10.0, 10.0],
        },
    ],
    experiment_name="test",
    objective_name="func",
    evaluation_function=eval_func,
    minimize=True,  # Optional, defaults to False.
    #parameter_constraints=["x1 + x2 <= 20"],  # Optional.
    #outcome_constraints=["l2norm <= 1.25"],  # Optional.
    total_trials=30, # Optional.
)

In [ ]:
best_parameters

In [ ]:
means, covariances = values
means

In [ ]:
import numpy as np

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.metrics.branin import branin
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import render, init_notebook_plotting

init_notebook_plotting()

In [ ]:
render(plot_contour(model=model, param_x='x', param_y='y', metric_name='func'))

In [ ]:
# `plot_single_method` expects a 2-d array of means, because it expects to average means from multiple 
# optimization runs, so we wrap out best objectives array in another array.
best_objectives = np.array([[trial.objective_mean for trial in experiment.trials.values()]])
best_objective_plot = optimization_trace_single_method(
    y=np.minimum.accumulate(best_objectives, axis=1),
    optimum=hartmann6.fmin,
    title="Model performance vs. # of iterations",
    ylabel="Hartmann6",
)
render(best_objective_plot)

In [ ]:
# data = exp.fetch_data()
gpei = Models.BOTORCH(experiment=exp, data=exp.eval())

In [ ]:
num_opt = cfg.bo.optimized
for i in range(num_opt):
    print(f"Running GP+EI optimization trial {i + 1}/{num_opt}...")
    # Reinitialize GP+EI model at each step with updated data.
    batch = exp.new_trial(generator_run=gpei.gen(1))
    gpei = Models.BOTORCH(experiment=exp, data=exp.eval())

## Tutorial: https://www.youtube.com/watch?v=BQ4kVn-Rt84

In [8]:
import os
import torch
import numpy as np
import plotly

In [9]:
def target_function(individuals):
  result = []
  for x in individuals:
    result.append(np.exp(-(x[0]-2)**2)+np.exp(-(x[0]-6)**2/10)+1/(x[0]**2+1))
  return torch.tensor(result)

In [10]:
import plotly.graph_objects as go

x=np.linspace(-2.,10.,100)
x_new=x.reshape((100,-1))
z=target_function(x_new)

data = go.Scatter(x=x,y=z,line_color="#FE73FF")

fig=go.Figure(data=data)
fig.update_layout(title="Example Function", xaxis_title="input", yaxis_title="output")
fig.show()

In [11]:
from scipy.optimize import minimize

def func(x):
  return -(np.exp(-(x-2)**2)+np.exp(-(x-6)**2/10)+1/(x**2+1))


minimize(func,x0=1)

      fun: -1.4018971812897256
 hess_inv: array([[0.57698765]])
      jac: array([-6.70552254e-07])
  message: 'Optimization terminated successfully.'
     nfev: 16
      nit: 4
     njev: 8
   status: 0
  success: True
        x: array([2.00087394])

In [12]:
import plotly.graph_objects as go

x=np.linspace(-2.,10.,100)
x_new=x.reshape((100,-1))
z=[func(x_[0]) for x_ in x_new]

data = go.Scatter(x=x,y=z,line_color="#FE73FF")

fig=go.Figure(data=data)
fig.update_layout(title="Example Function", xaxis_title="input", yaxis_title="output")
fig.show()

In [13]:
train_x=torch.rand(10,1)
train_x

tensor([[0.3855],
        [0.9607],
        [0.8412],
        [0.7151],
        [0.5639],
        [0.2916],
        [0.1702],
        [0.1133],
        [0.6085],
        [0.8582]])

In [14]:
exact_obj=target_function(train_x).unsqueeze(-1)
exact_obj

tensor([[0.9872],
        [0.9385],
        [0.9166],
        [0.9148],
        [0.9380],
        [1.0141],
        [1.0404],
        [1.0470],
        [0.9287],
        [0.9185]])

In [15]:
best_observed_value=exact_obj.max().item()
best_observed_value

1.0470372438430786

In [16]:
def generate_initial_data(n=10):
  train_x=torch.rand(n,1,dtype=torch.double)
  exact_obj=target_function(train_x).unsqueeze(-1)
  best_observed_value=exact_obj.max().item()
  return train_x,exact_obj,best_observed_value

In [17]:
init_x, init_y, best_init_y = generate_initial_data(20)
print(init_x.size())
print(init_y.size())

torch.Size([20, 1])
torch.Size([20, 1])


In [18]:
bounds = torch.tensor([[0.],[10.]])

In [19]:
from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood

single_model=SingleTaskGP(init_x,init_y)
mll=ExactMarginalLogLikelihood(single_model.likelihood,single_model)

In [20]:
from botorch import fit_gpytorch_model
fit_gpytorch_model(mll)

ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
  )
)

In [21]:
from botorch.acquisition.monte_carlo import qExpectedImprovement

EI = qExpectedImprovement(
    model=single_model,
    best_f=best_init_y
)


In [22]:
from botorch.optim import optimize_acqf
candidates, _ = optimize_acqf(
    acq_function=EI,
    bounds=bounds,
    q=10,
    num_restarts=200,
    raw_samples=1024,
    options={"batch_limit":5,"maxiter":200}
)
candidates

tensor([[ 6.3036],
        [ 3.7717],
        [ 4.6071],
        [ 8.0670],
        [ 2.8692],
        [ 7.1804],
        [ 2.0207],
        [10.0000],
        [ 5.5208],
        [ 9.0623]])

In [23]:
def get_next_points(init_x,init_y,best_init_y,bounds,n_points=1):
  single_model=SingleTaskGP(init_x,init_y)
  mll=ExactMarginalLogLikelihood(single_model.likelihood,single_model)
  fit_gpytorch_model(mll)
  
  EI = qExpectedImprovement(model=single_model, best_f=best_init_y)

  candidates, _ = optimize_acqf(
      acq_function=EI,
      bounds=bounds,
      q=n_points,
      num_restarts=200,
      raw_samples=1024,
      options={"batch_limit":5,"maxiter":200}
  )

  return candidates

In [24]:
n_runs=20

init_x, init_y,best_init_y = generate_initial_data(20)
bounds = torch.tensor([[0.],[10.]])

for i in range(n_runs):
    print(f"Nr. of optimization run: {i}")
    new_candidates = get_next_points(init_x, init_y, best_init_y, bounds, 1)
    new_results = target_function(new_candidates).unsqueeze(-1)

    print(f"New candidates are: {new_candidates}")
    init_x = torch.cat([init_x, new_candidates])
    init_y = torch.cat([init_y,new_results])

    best_init_y = init_y.max().item()
    print(f"Best point performs this way: {best_init_y}")

Nr. of optimization run: 0
New candidates are: tensor([[2.2991]])
Best point performs this way: 1.3277102708816528
Nr. of optimization run: 1
New candidates are: tensor([[2.8972]])
Best point performs this way: 1.3277102708816528
Nr. of optimization run: 2
New candidates are: tensor([[2.0212]])
Best point performs this way: 1.4015388488769531
Nr. of optimization run: 3
New candidates are: tensor([[10.]])
Best point performs this way: 1.4015388488769531
Nr. of optimization run: 4
New candidates are: tensor([[6.2417]])
Best point performs this way: 1.4015388488769531
Nr. of optimization run: 5
New candidates are: tensor([[7.4772]])
Best point performs this way: 1.4015388488769531
Nr. of optimization run: 6
New candidates are: tensor([[5.0098]])
Best point performs this way: 1.4015388488769531
Nr. of optimization run: 7
New candidates are: tensor([[1.9291]])
Best point performs this way: 1.4015388488769531
Nr. of optimization run: 8
New candidates are: tensor([[2.0179]])
Best point perfor